In [1]:
%matplotlib inline
from datetime import timedelta, datetime
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

from IPython.display import Video, HTML

import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature

from CloudFileHelper import *


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, matplotlib, xarray, cartopy

Software versions
Python 3.6.11 64bit [GCC Clang 11.0.0]
IPython 7.16.1
OS Darwin 21.3.0 x86_64 i386 64bit
numpy 1.19.2
matplotlib 3.3.2
xarray 0.16.2
cartopy 0.18.0
Mon Apr 11 20:30:27 2022 CEST

In [4]:
filedir = os.path.abspath(".")
filename = 'Particle_AZO_grid100000p_wtides_0601_hourly_MONTH_subregion.nc'

vdir = os.path.abspath(os.path.join(filedir, "..", "images"))
vfile = "anim_June_s1_TIDAL_all_subregion.mp4"
vfile_path = os.path.join(vdir, vfile)


In [20]:
if not os.path.exists(vfile_path):

    Writer = animation.writers['ffmpeg']
    ani_writer_nc = Writer(fps=15, metadata=dict(artist='LGomezNavarro'), bitrate=3200)
    
    get_from_surfdrive("https://surfdrive.surf.nl/files/index.php/s/ui1wS4BNTRpKJOz", os.path.join(filedir, filename))

    ds_subregion = xr.open_dataset(os.path.join(filedir, filename))
    
    # Defining time of simulation:
    outputdt = timedelta(hours=1)
    
    timerange = np.arange(np.datetime64('2010-06-01T00:30:00.000000000'), (np.datetime64('2010-06-15T00:30:00.000000000'))+np.timedelta64(outputdt), outputdt)  # timerange in nanoseconds
    #np.nanmin(ds_subregion['time'].values)
    # --> reduced end date from 2th June to 15th to show html animation below more easily
    
    fig = plt.figure(figsize=(14, 8))#, constrained_layout=True)

    ax1 = plt.subplot(111, projection=ccrs.PlateCarree())

    ax1.set_xlim([-32, -18])
    ax1.set_ylim([30, 40])

    resol = '10m'  # use data at this scale
    land = cartopy.feature.NaturalEarthFeature('physical', 'land', \
        scale=resol, edgecolor='k', facecolor=cfeature.COLORS['land'])
    ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', \
        scale=resol, edgecolor='none', facecolor=cfeature.COLORS['water'])

    ax1.add_feature(land, facecolor='burlywood', zorder=50) #beige
    ax1.add_feature(ocean, linewidth=0.2 )

    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-35, -30., -25., -20.])
    gl.ylocator = mticker.FixedLocator([30., 33., 36., 39.]) #np.arange(30., 42, 3)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.ylabel_style = {'size': 12}#, 'color': 'gray'}
    gl.xlabel_style = {'size': 12}


    time_id = np.where(ds_subregion['time'] == timerange[0]) # Indices of the data where time = 0

    scatter_subregion = ax1.scatter(ds_subregion['lon'].values[time_id], ds_subregion['lat'].values[time_id], s=1, c='r', transform=ccrs.PlateCarree(), zorder=20)

    t = str(timerange[0]) 
    title = plt.suptitle('Particles at t = ' + str(t[0:10]) + ' ' + str(t[11:16]), size=24)

    def animate(i):
        t = str(timerange[i])
        title.set_text('Particles at t = ' + str(t[0:10]) + ' ' + str(t[11:16]))

        time_id = np.where(ds_subregion['time'] == timerange[i])
        scatter_subregion.set_offsets(np.c_[ds_subregion['lon'].values[time_id], ds_subregion['lat'].values[time_id]])

    ax1.set_title('Tidal', size=20)

    anim = FuncAnimation(fig, animate, frames = len(timerange), interval=500)
    anim.save(vfile_path, writer=ani_writer_nc, dpi=92)
    
    plt.close()

In [21]:
Video(vfile_path, embed=True)

In [42]:
np.datetime64('2010-06-01T00:30:00.000000000')

numpy.datetime64('2010-06-01T00:30:00.000000000')

In [ ]:
# To show animation within a jupyter-notebook:

# HTML(anim.to_jshtml())

In [ ]:
# Set up formatting for the movie files and saving it:

# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# anim.save(rootdir + 'parcels_gallery/images/anim_June_s1_TIDAL_all_subregion.mp4', writer=writer, dpi=500)

In [39]:
import datetime

epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1e9

In [46]:
dt_ini = datetime.datetime(2010,6,1,0,30)

In [50]:
dt_ini_ns_since = unix_time_millis(dt_ini)

In [55]:
dt_ini_ns_since # float

1.2753522e+18

In [51]:
dts = ds_subregion.time[0,:].values + dt_ini_ns_since

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<m8[ns]') and dtype('float64')

In [73]:
86400000000000 / 1e9 / 3600

24.0

In [57]:
cc = ds_subregion.time[0,:].values

In [69]:
cc0 = cc[-1]

In [70]:
cc0

numpy.timedelta64(31536000000000000,'ns')

In [71]:
cc0 + np.datetime64('2010-06-01')

numpy.datetime64('2011-06-01T00:00:00.000000000')

In [ ]:
ds_subregion.time[0,:].values

In [49]:
ds_subregion.time[0,:].values

array([                0,    86400000000000,   172800000000000,
         259200000000000,   345600000000000,   432000000000000,
         518400000000000,   604800000000000,   691200000000000,
         777600000000000,   864000000000000,   950400000000000,
        1036800000000000,  1123200000000000,  1209600000000000,
        1296000000000000,  1382400000000000,  1468800000000000,
        1555200000000000,  1641600000000000,  1728000000000000,
        1814400000000000,  1900800000000000,  1987200000000000,
        2073600000000000,  2160000000000000,  2246400000000000,
        2332800000000000,  2419200000000000,  2505600000000000,
        2592000000000000,  2678400000000000,  2764800000000000,
        2851200000000000,  2937600000000000,  3024000000000000,
        3110400000000000,  3196800000000000,  3283200000000000,
        3369600000000000,  3456000000000000,  3542400000000000,
        3628800000000000,  3715200000000000,  3801600000000000,
        3888000000000000,  3974400000000

In [31]:
import datetime
seconds_since_epoch = .timestamp()

milliseconds_since_epoch = datetime.datetime.now().timestamp() * 1e9

In [32]:
milliseconds_since_epoch

1.649701013621281e+18

In [34]:
dt = datetime.datetime.fromtimestamp(1360287003083988472 // 1000000000)

In [35]:
dt

datetime.datetime(2013, 2, 8, 2, 30, 3)

In [36]:
s = dt.strftime('%Y-%m-%d %H:%M:%S')

In [37]:
s

'2013-02-08 02:30:03'

In [3]:
rootdir = "/Users/Gomez023/src/git/"

filedir = "parcels_gallery/data/"

ds_subregion_old = xr.open_dataset(rootdir + filedir + 'Particle_AZO_grid100000p_wtides_0601_hourly_MONTH_subregion.nc')

In [4]:
ds_subregion_old

<xarray.Dataset>
Dimensions:     (obs: 673, traj: 12657)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) float64 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float64 ...
    lon         (traj, obs) float64 ...
    z           (traj, obs) float64 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        2.2.2.dev126+g6dd05b7
    parcels_mesh:           spherical

In [5]:
ds_subregion_old.time.values

array([[                          'NaT',                           'NaT',
                                  'NaT', ...,
        '2010-06-28T22:30:00.000000000', '2010-06-28T23:30:00.000000000',
        '2010-06-29T00:30:00.000000000'],
       [                          'NaT',                           'NaT',
                                  'NaT', ...,
        '2010-06-28T22:30:00.000000000', '2010-06-28T23:30:00.000000000',
        '2010-06-29T00:30:00.000000000'],
       [                          'NaT',                           'NaT',
                                  'NaT', ...,
        '2010-06-28T22:30:00.000000000', '2010-06-28T23:30:00.000000000',
        '2010-06-29T00:30:00.000000000'],
       ...,
       [                          'NaT',                           'NaT',
                                  'NaT', ...,
                                  'NaT',                           'NaT',
                                  'NaT'],
       [                          'NaT',    

In [13]:
ds_subregion_old.time.values[-2,-1]

numpy.datetime64('2010-06-29T00:30:00.000000000')